In [48]:
import cv2
import pandas as pd
import cvlib as cv
import os 
from pathlib import Path
from utils.util import ensure_dir, prepare_device
from PIL import Image
import tqdm
import shutil

from utils.util import ensure_dir


def crop_coord(imagePath):
	image = cv2.imread(imagePath)
	face, confidence = cv.detect_face(image)
	if len(face)==0 :
		return 0
	x, y, w, h = face[0]
	coord = [x, y, w, h]

	return coord


def crop_face(coord_list, path, crop_path):
	if type(coord_list)==int:
		shutil.copy(path, crop_path)
		return
	x, y, w, h = coord_list
	image = cv2.imread(path)
	H, W, C = image.shape
	image_array = image[max(y-100,0):min(h+100,H), max(0,x-100):min(w+100,W)]
	cv2.imwrite(crop_path, image_array)
	

# # if __name__ == '__main__':
label_path = '/opt/ml/image-classification-level1-12/templates/data/train/train_with_label.csv'
df = pd.read_csv(label_path)
df['normal'] = df['path'].map(lambda x: 1 if 'normal' in x else 0)
df['crop_coor'] = df.apply(lambda x: crop_coord(x['path']) if x['normal']==1 else None, axis=1)
df = df.sort_values(by='name').fillna(method='backfill', limit=6).sort_index()
df['crop_path'] = df['path'].apply(lambda x: x.replace('images', 'image_crop'))
df.apply(lambda x: crop_face(x['crop_coor'], x['path'], x['crop_path']), axis=1)
df.to_csv(label_path, index=False)

In [27]:
eval_path = '/opt/ml/image-classification-level1-12/templates/data/eval'
eval_df = pd.read_csv(eval_path+'/info.csv')
eval_df['crop_coord'] = eval_df.apply(lambda x: crop_coord(eval_path+'/images/'+x['ImageID']),axis=1)

path = eval_path+'/images/'
crop_path = eval_path+'/crop_image/'

eval_df.apply(lambda x : crop_face(x['crop_coord'], path+x['ImageID'], crop_path+x['ImageID']), axis=1)

In [49]:
import pandas as pd

df = pd.read_csv('/opt/ml/image-classification-level1-12/templates/data/train/train_with_label.csv')
df_new = pd.DataFrame(df['path'].append(df['crop_path']))
df_new['label'] = df['label'].append(df['label'])
df_new.to_csv('/opt/ml/image-classification-level1-12/templates/data/train/train_with_crop.csv', index=False)

0        None
1        None
2        None
3        None
4        None
         ... 
12595    None
12596    None
12597    None
12598    None
12599    None
Length: 12600, dtype: object